<a href="https://colab.research.google.com/github/marquesgabi/Doutorado/blob/master/Copy_of_Features_Determination_teste_010_dez_06_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mahotas

     |████████████████████████████████| 1.5MB 2.7MB/s 
  Created wheel for mahotas: filename=mahotas-1.4.9-cp36-cp36m-linux_x86_64.whl size=4264056 sha256=27fb492149d389848ed48650173e1928b7a56ffb64eeeb57812db5ecce70ac27
  Stored in directory: /root/.cache/pip/wheels/02/e7/50/b67eee3b19c479cc5b41f10230bf9c1556be6eaaf095c6e997
Successfully built mahotas


In [0]:

!git clone https://github.com/ucfilho/marquesgabi #clonar do Github
%cd marquesgabi

Cloning into 'marquesgabi'...
remote: Enumerating objects: 343, done.
remote: Total 343 (delta 0), reused 0 (delta 0), pack-reused 343
Receiving objects: 100% (343/343), 8.36 MiB | 5.03 MiB/s, done.
Resolving deltas: 100% (194/194), done.
/content/marquesgabi


In [0]:
import numpy as np
import cv2
from skimage import data
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from skimage import io, util
import mahotas.features.texture as mht
import mahotas.features
import os 
import zipfile
from skimage import filters
from skimage import exposure
import skimage.feature as sk
from google.colab import files
from numpy import linalg as LA
from scipy.signal import find_peaks
from scipy.signal import peak_prominences
from scipy.signal import peak_widths
import re
import Go2BlackWhite
import Go2Mahotas
%matplotlib inline

In [0]:
Size=200 # tamanho da foto
Sub_Size=int(Size/5) # tamanho do fracionamento
Row_Crop=1/2 # posicao do corte
Crop=int(Size*Row_Crop)

In [0]:
!wget http://ftp.feq.ufu.br/ubirajara/GrBuInd.zip
Transfere='GrBuInd.zip'

--2019-12-10 13:05:11--  http://ftp.feq.ufu.br/ubirajara/GrBuInd.zip
Resolving ftp.feq.ufu.br (ftp.feq.ufu.br)... 200.19.144.11
Connecting to ftp.feq.ufu.br (ftp.feq.ufu.br)|200.19.144.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 497627 (486K) [application/zip]
Saving to: ‘GrBuInd.zip’

GrBuInd.zip         100%[===================>] 485.96K   197KB/s    in 2.5s    

2019-12-10 13:05:15 (197 KB/s) - ‘GrBuInd.zip’ saved [497627/497627]



In [0]:
file_name = zipfile.ZipFile(Transfere, 'r')
file_name.extractall()

In [0]:
# print(img_name)

In [0]:
# ww contain all photos in black and white
ww,img_name=Go2BlackWhite.BlackWhite(Transfere,Size)
# plt.axis("off")
# plt.imshow(ww[0], cmap = "gray")

In [0]:
# Find the list elements starting with specific letter
subs = 'B'
Buraco = [x for x in img_name if re.search(subs, x)] 
subs = 'G'
Grao = [x for x in img_name if re.search(subs, x)] 
subs = 'I'
Indef = [x for x in img_name if re.search(subs, x)] 

In [0]:
# find df texture for a given picture
df=Go2Mahotas.Mahotas(ww[0],Size,Sub_Size,Crop,img_name)
print(df)

          ASM    constrast    correl  ...  dif entropy      IMC1      IMC2
0    0.012146   839.716383  0.922992  ...     5.360705 -0.553436  0.999757
1    0.009309   852.210273  0.919052  ...     5.430678 -0.547121  0.999766
2    0.008023   864.961744  0.914335  ...     5.494037 -0.540839  0.999770
3    0.004495   885.729068  0.907571  ...     5.558871 -0.533217  0.999772
4    0.002481   910.649248  0.898534  ...     5.616679 -0.525523  0.999770
..        ...          ...       ...  ...          ...       ...       ...
156  0.008712  1573.804771  0.810473  ...     5.662041 -0.511908  0.999565
157  0.012512  1526.464033  0.824425  ...     5.580811 -0.518453  0.999551
158  0.018239  1504.009837  0.833059  ...     5.495274 -0.526079  0.999535
159  0.020161  1492.755909  0.839546  ...     5.417776 -0.533902  0.999531
160  0.024144  1481.756316  0.844719  ...     5.339334 -0.541121  0.999515

[161 rows x 13 columns]


In [0]:
j=0
cont=-1
Posicao_Buraco=[]
for i in img_name:
  cont=cont+1
  for k in Buraco:
    if(i==k):
      Posicao_Buraco.append(cont)
j=0
cont=-1
Posicao_Grao=[]
for i in img_name:
  cont=cont+1
  for k in Grao:
    if(i==k):
      Posicao_Grao.append(cont)

j=0
cont=-1
Posicao_Indef=[]
for i in img_name:
  cont=cont+1
  for k in Indef:
    if(i==k):
      Posicao_Indef.append(cont)

# print(Posicao_Grao)
# print(Posicao_Buraco)
# print(Posicao_Indef)


In [0]:
# compara constrast dos tres primeiros buracos
Num=len(Posicao_Buraco)
Matriz_X_Buraco=[]
#Num=3
for i in range(Num):
  k=Posicao_Buraco[i]
  df=Go2Mahotas.Mahotas(ww[k],Size,Sub_Size,Crop,img_name)
  Matriz_X_Buraco.append(np.array(df['constrast']).ravel())
  #plt.figure(i+1)
  #plt.plot(df['constrast'])
  #print(Matriz_X_Buraco)
Contraste_Buraco=pd.DataFrame(Matriz_X_Buraco)

In [0]:
# compara constrast dos graos
Num=len(Posicao_Grao)
Matriz_X_Grao=[]
for i in range(Num):
  k=Posicao_Grao[i]
  df=Go2Mahotas.Mahotas(ww[k],Size,Sub_Size,Crop,img_name)
  Matriz_X_Grao.append(np.array(df['constrast']).ravel())
  #plt.figure(i+1)
  #plt.plot(df['constrast'])
  #print(Matriz_X_Grao)
Contraste_Grao=pd.DataFrame(Matriz_X_Grao)

In [0]:
# compara constrast dos indefinidos
Num=len(Posicao_Indef)
Matriz_X_Indef=[]
for i in range(Num):
  k=Posicao_Indef[i]
  df=Go2Mahotas.Mahotas(ww[k],Size,Sub_Size,Crop,img_name)
  Matriz_X_Indef.append(np.array(df['constrast']).ravel())
  #plt.figure(i+1)
  #plt.plot(df['constrast'])
  #print(Matriz_X_Indef)
Contraste_Indef=pd.DataFrame(Matriz_X_Indef)

In [0]:
print(Contraste_Buraco.head())

         0          1          2    ...        158        159        160
0  10.217542  10.737327  11.381875  ...   2.519001   2.498003   2.473397
1   6.003546   5.834985   5.670657  ...   0.776007   0.768684   0.762829
2   0.931476   0.928858   0.926401  ...  11.728546  12.057861  12.352392
3  11.321692  11.112385  10.878456  ...   9.826890   9.810289   9.791580
4  22.274404  22.561366  22.652712  ...   9.063979   9.158033   9.172432

[5 rows x 161 columns]


In [0]:
print(Contraste_Grao.head())

          0           1           2    ...         158         159         160
0   55.587825   55.994613   56.367599  ...   52.978838   53.323603   53.671129
1   42.148796   41.217373   40.305153  ...   28.219461   28.475855   28.739082
2   68.178768   68.421474   68.673303  ...   20.419481   20.948352   21.496569
3  137.309698  137.241872  137.167020  ...  118.571741  116.425251  114.176524
4   15.956636   15.669235   15.386727  ...   33.118438   33.271832   33.426524

[5 rows x 161 columns]


In [0]:
print(Contraste_Indef.head())

         0          1          2    ...        158        159       160
0   0.610791   0.631797   0.651015  ...   2.068738   2.011719  1.964320
1   6.425748   6.820188   7.266938  ...  10.764859  10.548056  9.948919
2   1.440101   1.412044   1.380243  ...   5.825234   6.103789  6.373077
3  11.713519  11.476327  11.243212  ...   0.663774   0.650423  0.640841
4   1.030371   1.046318   1.059336  ...   2.526262   2.458814  2.393323

[5 rows x 161 columns]


In [0]:
# found drive
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
# get csv file
Contraste_Buraco.to_csv('Contraste_Buraco_01_dez_2019.csv')

In [0]:
# get csv file
Contraste_Buraco.to_csv('Contraste_Grao_01_dez_2019.csv')

In [0]:
# get csv file
Contraste_Buraco.to_csv('Contraste_Indef_01_dez_2019.csv')

In [0]:
# save in drive
!cp Contraste_Buraco_01_dez_2019.csv drive/My\ Drive/Maria_Gabriela_Textura_dados_dez_2019/

In [0]:
# save in drive
!cp Contraste_Grao_01_dez_2019.csv drive/My\ Drive/Maria_Gabriela_Textura_dados_dez_2019/

In [0]:
# save in drive
!cp Contraste_Indef_01_dez_2019.csv drive/My\ Drive/Maria_Gabriela_Textura_dados_dez_2019/